In [2]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import urllib.request, json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from tqdm import tqdm

Generic class to store an image and its IIIF representation

In [3]:
class Terzani_Photo(object):
    def __init__(self, iiif, country):
        self.iiif = iiif
        self.country = country
        
    def get_photo_link(self):
        return self.iiif["images"][0]["resource"]["@id"]

First we need to know all the collections for the Terzani archive. For this we scrape the HTML content from the Fondo Tiziano Terzani (collection 1352). We store the collections number in the ```collections``` list. We just need to discard collections 1353, 1360 and 1450 which are collections of collections

In [14]:
r = requests.get('http://dl.cini.it/collections/show/1353') # /ip: Returns the requester's IP Address.
soup = BeautifulSoup(r.text, "html.parser")
collections = []
unsupported_collections = ['1352', '1360', '1450']
for a in soup.find_all("a", href=True):
    link = a['href']
    if link.startswith("/collections/show/") and link[-4:] not in unsupported_collections:
        collections.append(link[-4:])
    
#Preview
collections[:5]
collections_country = dict.fromkeys(collections, None)
collections_country['1460'] = "China"
collections_country['1465'] = "China"
for i in range(1451, 1460):
    collections_country[str(i)] = "China"
collections_country['1444'] = "Japan"
collections_country['1404'] = "South Africa"
collections_country['1446'] = "Cambodia"
collections_country['1413'] = "India"
collections_country['1414'] = "India"
collections_country['1415'] = "India"
collections_country['1416'] = "India"
collections_country['1420'] = "India"
collections_country['1418'] = "South Korea"
collections_country['1419'] = "Laos"
collections_country['1421'] = "Philippines"
collections_country['1423'] = "Tibet"
collections_country['1425'] = "India"
collections_country['1424'] = "Nepal"
collections_country['1426'] = "Nepal"
collections_country['1427'] = "Singapore"
collections_country['1429'] = "Japan"
collections_country['1428'] = "Japan"
collections_country['1433'] = "Vietnam"
collections_country['1431'] = "Vietnam"
collections_country['1432'] = "Vietnam"
collections_country['1434'] = "Myanmar"
collections_country['1435'] = "Myanmar"
collections_country['1437'] = "Japan"
collections_country['1403'] = "Vietnam"
collections_country['1440'] = "Cambodia"
collections_country['1442'] = "Cambodia"
collections_country['1439'] = "Cambodia"
collections_country['1441'] = "Cambodia"
collections_country['1449'] = "Cambodia"
collections_country['1438'] = "Cambodia"

Now we can read the manifest of every collection. For every entry of the manifest, we only keep it if it is the recto of a photo

We check if an picture is color or black and white and store them seperately.

In [17]:
terzani_recto_iiif_color = list() #to store color photos in Terzani photo collection
terzani_recto_iiif_bw = list() #to store black and white photos in Terzani photo collection
terzani_recto_iiif = list()

threshold_color = 0.2

iiif_format = "/full/max/0/default.jpg"

for collection in tqdm(collections):
    url = "http://dl.cini.it/oa/collections/"+collection+"/manifest.json"

    response = urllib.request.urlopen(url)

    data = json.loads(response.read())

    for entry in data["sequences"][0]["canvases"]:
        if entry["label"] == None:
            continue
        if  entry["label"].lower().endswith("recto"):
            terzani_recto_iiif.append(Terzani_Photo(entry, collections_country[collection]))
            '''
            photo_link = entry["images"][0]["resource"]["service"]["@id"] + iiif_format
            photo = np.asarray(bytearray(urllib.request.urlopen(photo_link).read()), dtype="uint8")
            photo = cv2.imdecode(photo, cv2.IMREAD_COLOR)
            b,g,r = cv2.split(photo)
            isize = r.size
            rg = np.abs(r-g)
            rb = np.abs(r-b)
            gb = np.abs(g-b)
            diff =np.sum(rg+rb+gb)
            diff = diff / isize / (255 * 3)
            if diff > threshold_color: # As gray values have r-g = 0 and r-b = 0 and g-b = 0 diff will be near 0 for grayscale images and > 0 for colored images.

                terzani_recto_iiif_color.append(Terzani_Photo(entry, collections_country[collection]))
            else:
                terzani_recto_iiif_bw.append(Terzani_Photo(entry, collections_country[collection]))
            '''

100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:18<00:00,  3.11it/s]


In [5]:
len(terzani_recto_iiif_color)

1354

In [6]:
len(terzani_recto_iiif_bw)

7170

In [19]:
terzani_recto_iiif[0].country

'China'

Let's now dump this list in a pickle file

In [7]:
with open('terzani_recto_iiif_color.pickle', 'wb') as file:
    pickle.dump(terzani_recto_iiif_color, file)

In [8]:
with open('terzani_recto_iiif_bw.pickle', 'wb') as file:
    pickle.dump(terzani_recto_iiif_bw, file)

In [20]:
with open('terzani_recto_iiif.pickle', 'wb') as file:
    pickle.dump(terzani_recto_iiif, file)

In [4]:
imgs = pd.read_pickle('terzani_recto_iiif_bw.pickle')

In [16]:
collections_country

{'1460': 'China',
 '1465': 'China',
 '1451': 'China',
 '1452': 'China',
 '1453': 'China',
 '1454': 'China',
 '1455': 'China',
 '1456': 'China',
 '1457': 'China',
 '1458': 'China',
 '1459': 'China',
 '1393': None,
 '1405': None,
 '1406': None,
 '1407': None,
 '1443': None,
 '1408': None,
 '1409': None,
 '1410': None,
 '1411': None,
 '1444': 'Japan',
 '1445': None,
 '1446': 'Cambodia',
 '1412': None,
 '1404': 'South Africa',
 '1413': 'India',
 '1414': 'India',
 '1415': 'India',
 '1416': 'India',
 '1420': 'India',
 '1447': None,
 '1418': 'South Korea',
 '1417': None,
 '1419': 'Laos',
 '1448': None,
 '1421': 'Philippines',
 '1423': 'Tibet',
 '1422': None,
 '1425': 'India',
 '1424': 'Nepal',
 '1426': 'Nepal',
 '1427': 'Singapore',
 '1429': 'Japan',
 '1428': 'Japan',
 '1430': None,
 '1432': 'Vietnam',
 '1431': 'Vietnam',
 '1433': 'Vietnam',
 '1434': 'Myanmar',
 '1435': 'Myanmar',
 '1436': None,
 '1437': 'Japan',
 '1438': 'Cambodia',
 '1449': 'Cambodia',
 '1441': 'Cambodia',
 '1439': 'Cambodi